## Expired Futures
When searching for an expired asset, you will need to specify the date range based on the ExpiryDate property.

#### NuGet Packages

In [1]:
#r "nuget:Refinitiv.Data.Content, 1.0.0-beta3"
#r "nuget:Microsoft.Data.Analysis"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.Data.Analysis, 0.19.0 Refinitiv.Data.Content, 1.0.0-beta3

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

In [2]:
using Newtonsoft.Json.Linq;
using Refinitiv.Data.Content.SearchService;
using Refinitiv.Data.Core;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Data;
using System;
using Microsoft.DotNet.Interactive.Formatting;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

#### Table Output
Helper routine to output data in a table format.

In [3]:
Formatter.Register<DataTable>((dt, writer) =>
{
    if (dt.Columns.Count > 0)
    {
        var rows = new List<ICollection<IHtmlContent>>();

        // Process Each data row
        foreach(var row in dt.Rows.Cast<DataRow>())
        {
            var cells = new List<IHtmlContent>();
            foreach (var cell in row.ItemArray)
            {
                switch (cell)
                {
                    case null:
                        break;
                    case JValue val:
                        cells.Add(td(val.ToString()));
                        break;
                    case IEnumerable<JToken> list:
                        cells.Add(td($"[{string.Join(", ", list.Select(node => $"{node}"))}]"));
                        break;
                    default:
                        cells.Add(td(cell));
                        break;
                }                
            }
            rows.Add(cells);
        }
        
        // Header
        var headers = new List<IHtmlContent>();
        foreach (DataColumn col in dt.Columns)
            headers.Add(th(col.ColumnName) as IHtmlContent);

        var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
        writer.Write(t);
    }
}, "text/html");

In [4]:
// Create a session into the desktop
DesktopSession.Definition().GetSession().OnState((s, state, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (State: {state})"))
                                        .OnEvent((s, eventCode, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (Event: {eventCode})"))
                                        .Open();

24/01/2022 11:55:14 AM:DesktopSession. (State: DesktopSession is Pending)
24/01/2022 11:55:15 AM:DesktopSession. (Event: {
  "Contents": "Desktop Session Successfully Authenticated"
})
24/01/2022 11:55:15 AM:DesktopSession. (State: DesktopSession is Opened)


#### Example - Expired Vodafone futures in the month of Nov 2019.

In [5]:
var filter = "TickerSymbol eq 'vod' and ExpiryDate lt 2019-12-01 and ExpiryDate ge 2019-11-01";
var select = "DTSubjectName, ExchangeName, RIC, RCSAssetCategoryLeaf, AssetState, ExpiryDate, UnderlyingQuoteRIC";

var response = Search.Definition(Search.View.DerivativeQuotes).Filter(filter)
                                                              .Select(select)
                                                              .GetData();
response.Data.Table

Johannesburg Stock Exchange Vodacom Group SSF Equity Future Nov 2019,Johannesburg Stock Exchange,VODQX9^1,Equity Future,DC,21/11/2019 12:00:00 AM,[VODJ.J]
Johannesburg Stock Exchange Vodacom Group Limited Cash Settled SSF Equity Future Nov 2019,Johannesburg Stock Exchange,VODSX9^1,Equity Future,DC,21/11/2019 12:00:00 AM,[VODJ.J]


#### Example - Expired Crude Oil Futures in the last year

In [6]:
// Build expiry date range expression
var dateOneYearAgo = DateTime.Now.AddYears(-1);
var dateRange = $"ExpiryDate ge {dateOneYearAgo:yyyy-MM-dd} and ExpiryDate lt {DateTime.Now:yyyy-MM-dd}";

In [7]:
filter = $@"RCSUnderlyingProductName xeq 'Energy' and
            SearchAllCategoryv3 eq 'Commodities' and
            RCSAssetCategoryName eq 'Derivative\Future' and RCSUnderlyingProductName eq 'Crude Oil' and
            ExchangeName xeq 'Source is a Refinitiv Contributor' and {dateRange}";

response = Search.Definition(Search.View.SearchAll).Filter(filter)
                                                   .OrderBy("ExpiryDate desc")
                                                   .Top(30)
                                                   .Select("DTSubjectName, RIC, ExpiryDate, RCSUnderlyingProductLeaf")
                                                   .GetData();
response.Data.Table

ICE Crude Oil Average Monthly,LCOAVGMU21,30/09/2021 12:00:00 AM,Crude Oil
ICE Crude Oil Average Quarterly,LCOAVGQU21,30/09/2021 12:00:00 AM,Crude Oil
NYMEX Light Sweet Crude Oil (WTI) Composite Energy Future Average Quarterly,CLAVGQU21,30/09/2021 12:00:00 AM,Crude Oil
TOCOM Crude Oil Full Session Energy Future Average Monthly,JCOAVGMU21,30/09/2021 12:00:00 AM,Crude Oil
TOCOM Crude Oil Full Session Energy Future Average Quarterly,JCOAVGQU21,30/09/2021 12:00:00 AM,Crude Oil
NYMEX Light Sweet Crude Oil (WTI) Composite Energy Future Average Monthly,CLAVGMU21,30/09/2021 12:00:00 AM,Crude Oil
ICE WTI Crude Futures Electronic Energy Future Average Monthly,WTCLAVGMU21,30/09/2021 12:00:00 AM,Crude Oil
ICE WTI Crude Futures Electronic Energy Future Average Quarterly,WTCLAVGQU21,30/09/2021 12:00:00 AM,Crude Oil
BFO Crude Oil Dated Swap Europe Average Quarterly,TBFODTDAVGQU21,30/09/2021 12:00:00 AM,Crude Oil
Dubai Crude Oil Swap Asia Average Quarterly,TDUBSGSWAVGQU21,30/09/2021 12:00:00 AM,Crude Oil
BFO Crude Oil Dated Swap Europe Average Monthly,TBFODTDAVGMU21,30/09/2021 12:00:00 AM,Crude Oil


#### Example - Expired Electricity Futures in the last year

In [8]:
filter = $@"SearchAllCategoryv3 eq 'Commodities' and
            RCSUnderlyingProductName eq 'Power' and
            ExchangeName xeq 'Source is a Refinitiv Contributor' and {dateRange}";

response = Search.Definition(Search.View.SearchAll).Filter(filter)
                                                   .OrderBy("ExpiryDate desc")
                                                   .Top(30)
                                                   .Select("DTSubjectName, RIC, ExpiryDate, RCSUnderlyingProductLeaf")
                                                   .GetData();
response.Data.Table

Power Italy Baseload Fair Value Monthly Dec 2021,TRITBFVMZ1,30/11/2021 12:00:00 AM,Baseload
Power Germany Baseload Fair Value Monthly Dec 2021,TRDEBFVMZ1,30/11/2021 12:00:00 AM,Baseload
Power Nordic Sundsvall Fair Value Monthly Dec 2021,TRSUNBFVMZ1,30/11/2021 12:00:00 AM,Power
Power Nordic TROMSO Fair Value Monthly Dec 2021,TRTROBFVMZ1,30/11/2021 12:00:00 AM,Power
Power UK Baseload Fair Value Monthly Dec 2021,TRGBBFVMZ1,30/11/2021 12:00:00 AM,Baseload
Power Nordic Oslo Fair Value Monthly Dec 2021,TROSLBFVMZ1,30/11/2021 12:00:00 AM,Power
Power Nordic Baseload Fair Value Monthly Dec 2021,TRNOBFVMZ1,30/11/2021 12:00:00 AM,Baseload
Power Nordic Stockholm Fair Value Monthly Dec 2021,TRSTOBFVMZ1,30/11/2021 12:00:00 AM,Power
RFV JP TYO Power Fair Value Monthly Dec 2021,TRJPTYOBFVMZ1,30/11/2021 12:00:00 AM,Power
RFV JP KAN Power Fair Value Balance of Month Nov 2021,TRJPKANBFVBMX1,30/11/2021 12:00:00 AM,Power
Power Swiss Baseload Fair Value Monthly Dec 2021,TRCHBFVMZ1,30/11/2021 12:00:00 AM,Baseload
